In [1]:
# Setup Libraries
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.figure_factory as ff
from datetime import datetime, timedelta
import datetime
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Setup Dataset Path
DATA_RSRP_PATH = '../data/processed/RSRP_processed.parquet'
DATA_TRAFFIC_PATH = '../data/processed/TrafficVolume_processed.parquet'

In [3]:
#Load Data
df_rsrp = pd.read_parquet(DATA_RSRP_PATH)
df_traffic = pd.read_parquet(DATA_TRAFFIC_PATH)

### We will Choose data that related to Operator A from two above Dataset then join two dataset

In [4]:
#Select Rows that related to Operator A from all Dataset
df_rsrp_operator_a = df_rsrp[df_rsrp['RadioOperatorName']=='Operator A']
df_traffic_operator_a = df_traffic[df_traffic['RadioOperatorName']=='Operator A']

In [5]:
print(df_rsrp_operator_a.shape)
print(df_traffic_operator_a.shape)

(1406266, 11)
(68171, 10)


In [6]:
#drop columns that not used in traffic dataset
df_traffic_operator_a = df_traffic_operator_a.drop(['RadioConnectionType', 'Country', 'RadioNetworkGeneration', 'RadioOperatorName','RadioMobileDataEnabled'], axis=1)

In [7]:
#Select rows from RSRP Dataset where location founded in traffic volume Dataset
df_traffic_operator_a['conc'] = df_traffic_operator_a['LocationLatitude'].map(str) + ' - '+df_traffic_operator_a['LocationLongitude'].map(str)
df_rsrp_operator_a['conc'] = df_rsrp_operator_a['LocationLatitude'].map(str) + ' - '+df_rsrp_operator_a['LocationLongitude'].map(str)
df_join_rsrp_location = df_rsrp_operator_a[df_rsrp_operator_a['conc'].isin(df_traffic_operator_a['conc'].unique())]

In [8]:
df_join_rsrp_location.head(1)

,Timestamp,LocationLatitude,LocationLongitude,RadioConnectionType,Country,RadioNetworkGeneration,RadioOperatorName,RSRP,RadioMobileDataEnabled,DeviceManufacturer,DeviceName,conc
62,2019-11-02 01:39:40,24.691517,46.65258,Mobile,Saudi Arabia,4G,Operator A,-94,Enabled,HUAWEI,Huawei Y5 2019,24.691517 - 46.65258


In [9]:
df_traffic_operator_a.head(1)

,Timestamp,LocationLatitude,LocationLongitude,TrafficDirection,TrafficVolume,conc
0,2019-11-02 00:45:00,24.636574,46.710026,Downlink,14.95763,24.636574 - 46.710026


In [10]:
# Round Time Function
def round_time(dt=None, round_to=60*15):
    if dt == None: 
        dt = datetime.datetime.now()
    seconds = (dt - dt.min).seconds
    rounding = (seconds+round_to/2) // round_to * round_to
    return dt + datetime.timedelta(0,rounding-seconds,-dt.microsecond)

In [11]:
#Select rows based on time approxmate
df_join_rsrp_location['time_approxmate'] = df_join_rsrp_location['Timestamp'].apply(lambda x : round_time(x.to_pydatetime()))
df_traffic_operator_a['time_approxmate'] = df_traffic_operator_a['Timestamp']

In [12]:
df_join_rsrp_location.head(1)

,Timestamp,LocationLatitude,LocationLongitude,RadioConnectionType,Country,RadioNetworkGeneration,RadioOperatorName,RSRP,RadioMobileDataEnabled,DeviceManufacturer,DeviceName,conc,time_approxmate
62,2019-11-02 01:39:40,24.691517,46.65258,Mobile,Saudi Arabia,4G,Operator A,-94,Enabled,HUAWEI,Huawei Y5 2019,24.691517 - 46.65258,2019-11-02 01:45:00


In [13]:
df_traffic_operator_a.head(1)

,Timestamp,LocationLatitude,LocationLongitude,TrafficDirection,TrafficVolume,conc,time_approxmate
0,2019-11-02 00:45:00,24.636574,46.710026,Downlink,14.95763,24.636574 - 46.710026,2019-11-02 00:45:00


In [14]:
# Calculate mean value of RSRP for each timestamp
df_join_rsrp_location['conctime'] = df_join_rsrp_location['conc'].map(str) + ' - ' + df_join_rsrp_location['time_approxmate'].map(str)
snap_mean = df_join_rsrp_location[['conctime','RSRP']]
snap_mean = snap_mean.groupby('conctime').mean().reset_index()

In [15]:
# Add mean value of rsrp to df_join_rsrp_location
df_join_rsrp_location = pd.merge(df_join_rsrp_location,snap_mean, on=['conctime'])
df_join_rsrp_location.head(1)

,Timestamp,LocationLatitude,LocationLongitude,RadioConnectionType,Country,RadioNetworkGeneration,RadioOperatorName,RSRP_x,RadioMobileDataEnabled,DeviceManufacturer,DeviceName,conc,time_approxmate,conctime,RSRP_y
0,2019-11-02 01:39:40,24.691517,46.65258,Mobile,Saudi Arabia,4G,Operator A,-94,Enabled,HUAWEI,Huawei Y5 2019,24.691517 - 46.65258,2019-11-02 01:45:00,24.691517 - 46.65258 - 2019-11-02 01:45:00,-93.444444


In [16]:
result = pd.merge(df_join_rsrp_location, df_traffic_operator_a, on=["conc",'time_approxmate'])
result.head(1)

,Timestamp_x,LocationLatitude_x,LocationLongitude_x,RadioConnectionType,Country,RadioNetworkGeneration,RadioOperatorName,RSRP_x,RadioMobileDataEnabled,DeviceManufacturer,DeviceName,conc,time_approxmate,conctime,RSRP_y,Timestamp_y,LocationLatitude_y,LocationLongitude_y,TrafficDirection,TrafficVolume
0,2019-11-02 01:39:40,24.691517,46.65258,Mobile,Saudi Arabia,4G,Operator A,-94,Enabled,HUAWEI,Huawei Y5 2019,24.691517 - 46.65258,2019-11-02 01:45:00,24.691517 - 46.65258 - 2019-11-02 01:45:00,-93.444444,2019-11-02 01:45:00,24.691517,46.65258,Downlink,6.218721


In [17]:
#Drop unneeded columns
result.drop(['conc','conctime','Timestamp_y','LocationLatitude_y',
             'LocationLongitude_y','Country','RadioOperatorName'],axis=1,inplace=True)
result.head(1)

,Timestamp_x,LocationLatitude_x,LocationLongitude_x,RadioConnectionType,RadioNetworkGeneration,RSRP_x,RadioMobileDataEnabled,DeviceManufacturer,DeviceName,time_approxmate,RSRP_y,TrafficDirection,TrafficVolume
0,2019-11-02 01:39:40,24.691517,46.65258,Mobile,4G,-94,Enabled,HUAWEI,Huawei Y5 2019,2019-11-02 01:45:00,-93.444444,Downlink,6.218721


In [18]:
#Rename some columns
result.rename(columns ={'Timestamp_x':'Timestamp','LocationLatitude_x':'LocationLatitude',
                        'LocationLongitude_x':'LocationLongitude','RSRP_y':'Average_RSRP','RSRP_x':'RSRP'},inplace=True)
result.head(1)

,Timestamp,LocationLatitude,LocationLongitude,RadioConnectionType,RadioNetworkGeneration,RSRP,RadioMobileDataEnabled,DeviceManufacturer,DeviceName,time_approxmate,Average_RSRP,TrafficDirection,TrafficVolume
0,2019-11-02 01:39:40,24.691517,46.65258,Mobile,4G,-94,Enabled,HUAWEI,Huawei Y5 2019,2019-11-02 01:45:00,-93.444444,Downlink,6.218721


In [19]:
# Show uniqe values of each column
for i in range(result.shape[1]):
    print("###########################     {}    ###########################".format(result.columns[i]))
    print(result.iloc[:,i].unique())
    print(" ")
    print(" ")

###########################     Timestamp    ###########################
['2019-11-02T01:39:40.000000000' '2019-11-02T01:41:59.000000000'
 '2019-11-02T01:41:54.000000000' ... '2019-11-04T19:27:52.000000000'
 '2019-11-04T19:28:57.000000000' '2019-11-03T08:05:32.000000000']
 
 
###########################     LocationLatitude    ###########################
[24.691517 24.618221 24.751925 ... 24.523375 24.812092 24.814854]
 
 
###########################     LocationLongitude    ###########################
[46.65258  46.72937  46.8578   ... 46.752865 46.753765 46.752903]
 
 
###########################     RadioConnectionType    ###########################
['Mobile' 'WiFi' 'Unknown']
 
 
###########################     RadioNetworkGeneration    ###########################
['4G' '3G' '2G']
 
 
###########################     RSRP    ###########################
[ -94  -89  -95  -91  -93  -97 -140  -79  -87  -96  -78 -100  -83  -92
  -98  -72  -81  -86  -82  -63  -85  -68  -67  -80  -90  -88 

In [20]:
result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 79614 entries, 0 to 79613
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Timestamp               79614 non-null  datetime64[ns]
 1   LocationLatitude        79614 non-null  float64       
 2   LocationLongitude       79614 non-null  float64       
 3   RadioConnectionType     79614 non-null  object        
 4   RadioNetworkGeneration  79614 non-null  object        
 5   RSRP                    79614 non-null  int64         
 6   RadioMobileDataEnabled  79614 non-null  object        
 7   DeviceManufacturer      79614 non-null  object        
 8   DeviceName              79614 non-null  object        
 9   time_approxmate         79614 non-null  datetime64[ns]
 10  Average_RSRP            79614 non-null  float64       
 11  TrafficDirection        79614 non-null  object        
 12  TrafficVolume           79614 non-null  float6

In [21]:
# Missing Values
total = result.isnull().sum().sort_values(ascending=False)
percent = (result.isnull().sum()/result.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(25)

,Total,Percent
Timestamp,0,0.0
LocationLatitude,0,0.0
LocationLongitude,0,0.0
RadioConnectionType,0,0.0
RadioNetworkGeneration,0,0.0
RSRP,0,0.0
RadioMobileDataEnabled,0,0.0
DeviceManufacturer,0,0.0
DeviceName,0,0.0
time_approxmate,0,0.0


In [22]:
#check duplicate rows
def check_duplicate(data):
    duplicate = data.duplicated()
    unique_data  = pd.Series(data.shape[0] - duplicate.value_counts())
    duplicate_data = pd.Series(data.shape[0] - duplicate.value_counts()[0])
    unique_data_percent = pd.Series((unique_data/data.shape[0])*100)
    duplicate_data_percent = pd.Series((duplicate_data/data.shape[0])*100)
    dub = pd.concat([unique_data, duplicate_data,unique_data_percent,duplicate_data_percent], axis=1, keys=['Unique Count', 'Duplicate Count','Unique percent','Duplicate Percent'])
    return dub

check_duplicate(result)

,Unique Count,Duplicate Count,Unique percent,Duplicate Percent
False,0,0,0.0,0.0


In [23]:
# Save File
result.to_parquet('../data/processed/RSRP_Traffic.parquet')